<a href="https://colab.research.google.com/github/castudil/bacteria-multi-label/blob/main/multilabel_bac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries used

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import ClassifierChain
from sklearn.metrics import (f1_score, multilabel_confusion_matrix,
                             accuracy_score, hamming_loss, jaccard_score, make_scorer)

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

import xgboost as xgb

from joblib import dump, load
import joblib

In [2]:
os.chdir("..")

In [3]:
train_file = "data/processed/raw/train_s_aureus_driams.csv"
train_bac = pd.read_csv(train_file)
train_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9993,9994,9995,9996,9997,9998,9999,Oxacillin,Clindamycin,Fusidic acid
0,0.018721,0.016147,0.016983,0.021218,0.020846,0.019784,0.019405,0.023356,0.026224,0.026569,...,0.037966,0.030364,0.037545,0.040851,0.034176,0.046110,0.025638,0.0,0.0,0.0
1,0.009001,0.007475,0.006874,0.008575,0.009539,0.007894,0.008314,0.008013,0.008664,0.008923,...,0.014496,0.024966,0.027437,0.026541,0.022940,0.020572,0.032504,0.0,0.0,0.0
2,0.022354,0.020220,0.020910,0.024631,0.021436,0.021197,0.020229,0.018818,0.018637,0.018815,...,0.024620,0.022942,0.026715,0.032045,0.030431,0.029085,0.013117,0.0,0.0,0.0
3,0.017619,0.016073,0.016407,0.018011,0.019364,0.018950,0.017607,0.019116,0.023623,0.024492,...,0.051312,0.047458,0.049338,0.055039,0.054541,0.058643,0.058919,0.0,0.0,0.0
4,0.008264,0.008229,0.006753,0.006657,0.010107,0.007039,0.008250,0.010670,0.008134,0.006513,...,0.236769,0.217499,0.187244,0.216243,0.221910,0.226531,0.221965,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,0.056616,0.039011,0.040380,0.048517,0.050865,0.047771,0.049312,0.048257,0.049417,0.049000,...,0.021169,0.023617,0.033694,0.021037,0.018727,0.010641,0.009238,0.0,1.0,0.0
2820,0.125837,0.107712,0.109186,0.107613,0.109855,0.105060,0.099640,0.104169,0.120303,0.125067,...,0.082375,0.083446,0.096510,0.084883,0.092228,0.085599,0.042142,0.0,1.0,0.0
2821,0.000000,0.000000,0.035603,0.039994,0.042372,0.046666,0.045781,0.043914,0.039875,0.037170,...,0.006903,0.008322,0.011071,0.010274,0.004682,0.003547,0.001744,0.0,0.0,0.0
2822,0.005443,0.005998,0.003670,0.005588,0.006124,0.005019,0.004853,0.005400,0.004169,0.005151,...,0.049241,0.039586,0.050542,0.039139,0.046816,0.043036,0.037402,1.0,1.0,0.0


In [4]:
test_file = "data/processed/raw/test_s_aureus_driams.csv"
test_bac = pd.read_csv(test_file)
test_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9993,9994,9995,9996,9997,9998,9999,Oxacillin,Clindamycin,Fusidic acid
0,0.044453,0.032486,0.032540,0.034223,0.037528,0.039503,0.031378,0.035506,0.037688,0.035658,...,0.247814,0.263833,0.279904,0.264432,0.241573,0.266020,0.231517,0.0,0.0,0.0
1,0.004318,0.001881,0.001274,0.000902,0.000892,0.000049,0.002188,0.002001,0.003081,0.003384,...,0.033364,0.042735,0.066426,0.057485,0.052903,0.050839,0.036631,0.0,0.0,0.0
2,0.026184,0.026459,0.025393,0.028609,0.031314,0.031739,0.033337,0.028051,0.028047,0.028978,...,0.086746,0.087719,0.094103,0.080969,0.073970,0.069047,0.070988,0.0,1.0,0.0
3,0.000000,0.015010,0.017782,0.014582,0.015084,0.018046,0.014461,0.014400,0.018769,0.018272,...,0.005062,0.006748,0.004573,0.007583,0.008427,0.004729,0.002394,0.0,0.0,1.0
4,0.060499,0.043034,0.030296,0.032635,0.031272,0.032762,0.031154,0.030382,0.030233,0.029438,...,0.029452,0.033288,0.039230,0.046477,0.037219,0.022227,0.019920,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.013092,0.011730,0.009902,0.013513,0.015317,0.011370,0.013338,0.010117,0.010994,0.009222,...,0.050851,0.046334,0.042840,0.061644,0.066948,0.061007,0.056178,0.0,0.0,0.0
703,0.021222,0.015896,0.015512,0.017995,0.018663,0.019427,0.018667,0.014589,0.016765,0.015071,...,0.047285,0.056455,0.058002,0.055528,0.043539,0.039962,0.034761,1.0,0.0,0.0
704,0.045613,0.041040,0.046177,0.050216,0.046525,0.048843,0.045920,0.044283,0.043983,0.046330,...,0.104234,0.094692,0.090012,0.096624,0.092228,0.100024,0.043682,0.0,0.0,1.0
705,0.015193,0.011922,0.010877,0.009975,0.010474,0.012171,0.010417,0.010783,0.013170,0.014157,...,0.051772,0.058255,0.074609,0.068249,0.049157,0.070229,0.043615,1.0,0.0,0.0


In [5]:
train_x = train_bac[train_bac.columns.drop(list(train_bac.filter(regex='[^0-9]')))]
test_x = test_bac[test_bac.columns.drop(list(test_bac.filter(regex='[^0-9]')))]

In [6]:
antibiotics = train_bac.columns.drop(train_x.columns)

In [7]:
train_y = train_bac[antibiotics]
test_y = test_bac[antibiotics]

In [8]:
def multilabel_f1_wrapper(true, pred, average="weighted"):
    if isinstance(true, list):
        true = np.array(true)
    elif isinstance(true, pd.DataFrame):
        true = true.to_numpy()
    if isinstance(pred, list):
        pred = np.array(pred)
    elif isinstance(true, pd.DataFrame):
        pred = pred.to_numpy()
    column = 0
    total = 0
    while column < true[0].size:
        total+=f1_score(true[:, column], pred[:, column], average=average)
        column+=1
    return total/(column)

In [9]:
def report(true, pred):
        
    hl = hamming_loss(true, pred)
    f1w = multilabel_f1_wrapper(true, pred, "weighted")
    acc = accuracy_score(true, pred)
    
    f1u = multilabel_f1_wrapper(true, pred, "macro")
    f1su = f1_score(true, pred, average="macro")
    f1sw = f1_score(true, pred, average="weighted")

    
    print("Main metrics:")
    print(" Hamming Loss:", hl)
    print(" Accuracy:", acc)
    print(" F1 Score (Weighted):", f1w)
    print("================================================")
    print("Other metrics:")
    print(" F1 Score (Unweighted):", f1u)
    print(" F1 Score (sklearn Unweighted):", f1su)
    print(" F1 Score (sklearn Weighted):", f1sw)
    return hl, acc, f1w

___


In [10]:
bayesopt = BayesSearchCV(
    ClassifierChain(xgb.XGBClassifier(), random_state=0),
    {
        "base_estimator__objective": Categorical(["binary:logistic"]),
        "base_estimator__max_depth": Integer(1, 10),
        "base_estimator__min_child_weight": Real(1e-6, 10, prior="log-uniform"),
        "base_estimator__max_delta_step": Real(1e-6, 10, prior="log-uniform"),
        "base_estimator__subsample": Real(1e-6, 1, prior="log-uniform"),
        "base_estimator__tree_method": Categorical(["exact", "approx", "hist"]),
        "base_estimator__scale_pos_weight": Real(1e-6, 10, prior="log-uniform"),
        "base_estimator__gamma": Real(1e-6, 10, prior="log-uniform"),
        "base_estimator__eta": Real(1e-6, 1, prior="log-uniform")
    },
    n_iter=250,
    cv=5,
    random_state=0,
    n_jobs=1,
    n_points=1,
    scoring=make_scorer(multilabel_f1_wrapper),
    verbose=1,
)

bayesopt.fit(train_x, train_y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

BayesSearchCV(cv=5,
              estimator=ClassifierChain(base_estimator=XGBClassifier(base_score=None,
                                                                     booster=None,
                                                                     callbacks=None,
                                                                     colsample_bylevel=None,
                                                                     colsample_bynode=None,
                                                                     colsample_bytree=None,
                                                                     early_stopping_rounds=None,
                                                                     enable_categorical=False,
                                                                     eval_metric=None,
                                                                     feature_types=None,
                                                                     gamma=None,
                                                                     gpu_id=None,
                                                                     grow_policy=None,
                                                                     importance_type=None,
                                                                     interaction_cons...
                             'base_estimator__objective': Categorical(categories=('binary:logistic',), prior=None),
                             'base_estimator__scale_pos_weight': Real(low=1e-06, high=10, prior='log-uniform', transform='normalize'),
                             'base_estimator__subsample': Real(low=1e-06, high=1, prior='log-uniform', transform='normalize'),
                             'base_estimator__tree_method': Categorical(categories=('exact', 'approx', 'hist'), prior=None)},
              verbose=1)

In [11]:
best_iteration = 0
for i in range(0, 250):
    if bayesopt.cv_results_["mean_test_score"][i] == bayesopt.best_score_:
        best_iteration = i
print("Best iteration:", best_iteration)
print("Split scores:")
for i in range(0, 5):
    print("", i, bayesopt.cv_results_["split"+str(i)+"_test_score"][best_iteration])
    
print("Mean score:", bayesopt.best_score_)
print("Best parameter combination found:", bayesopt.best_params_)

Best iteration: 227
Split scores:
 0 0.820343579865027
 1 0.8369469377938566
 2 0.8299130604982062
 3 0.8324157502908109
 4 0.8283769438981426
Mean score: 0.8295992544692087
Best parameter combination found: OrderedDict([('base_estimator__eta', 0.01800129117578471), ('base_estimator__gamma', 0.024282066345515534), ('base_estimator__max_delta_step', 0.0037650147666433023), ('base_estimator__max_depth', 8), ('base_estimator__min_child_weight', 8.32868910859238), ('base_estimator__objective', 'binary:logistic'), ('base_estimator__scale_pos_weight', 3.084189256760568), ('base_estimator__subsample', 0.14890444759709648), ('base_estimator__tree_method', 'exact')])


In [12]:
model_file = "xgb_s_aureus_raw.joblib"

In [15]:
dump(bayesopt.best_estimator_, model_file) 

['xgb_s_aureus_raw.joblib']

In [18]:
model = load(model_file)

In [19]:
pred = model.predict(test_x)
model_hl, model_acc, model_f1 = report(test_y, pred)

Main metrics:
 Hamming Loss: 0.13625648279113625
 Accuracy: 0.6605374823196606
 F1 Score (Weighted): 0.8323409200315045
Other metrics:
 F1 Score (Unweighted): 0.5495689021605566
 F1 Score (sklearn Unweighted): 0.17572362278244627
 F1 Score (sklearn Weighted): 0.24274180260339429


In [ ]:
fig, axes = plt.subplots(1, len(antibiotics), figsize=(len(antibiotics)*5, 5))
fig.supxlabel("Predicted Label")
fig.supylabel("True Label")

cm_svm_c = multilabel_confusion_matrix(test_y, (pred > 0.5))

for i in range(len(antibiotics)):
  sns.heatmap(ax=axes[i], data=cm_svm_c[i], annot=True, fmt='d', cbar=None, cmap="Blues", xticklabels=["S", "R"], yticklabels=["S", "R"]).set(title=antibiotics[i])

In [ ]:
proba = model.predict_proba(test_x)
proba

In [ ]:
for antibiotic in range(len(antibiotics)):
    count_tp = 0
    count_tn = 0
    count_fp = 0
    count_fn = 0
    sum_tp = 0
    sum_tn = 0
    sum_fp = 0
    sum_fn = 0
    for i in range(len(proba[:, antibiotic])):
        disc_pred = int(proba[i, antibiotic] > 0.5)
        if disc_pred == 1:
            if disc_pred == test_y.iloc[i, antibiotic]:
                count_tp += 1
                sum_tp += proba[i, antibiotic]
            else:
                count_fp += 1
                sum_fp += proba[i, antibiotic]
        else:
            if disc_pred == test_y.iloc[i, antibiotic]:
                count_tn += 1
                sum_tn += proba[i, antibiotic]
            else:
                count_fn += 1
                sum_fn += proba[i, antibiotic]
    print("Results for antibiotic", antibiotics[antibiotic])
    if count_tp == 0:
        print(" Mean TP: None")
    else: 
        print(" Mean TP:", sum_tp/count_tp)
    if count_tn == 0:
        print(" Mean TN: None")
    else: 
        print(" Mean TN:", sum_tn/count_tn)
    if count_fp == 0:
        print(" Mean FP: None")
    else: 
        print(" Mean FP:", sum_fp/count_fp)
    if count_fn == 0:
        print(" Mean FN: None")
    else: 
        print(" Mean FN:", sum_fn/count_fn)
